In [ ]:
import json

import numpy as np

from conf.user_info import USER_EMAIL_ID

"""
certain things are needed to be cared
names of sender and depts are important
remove mails sent by you
don't consider any default label

"""
import joblib
from time import time
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
t0 = time()
df = pd.read_csv('../data/training_data.csv', sep='~', index_col=0)
print(f'done in {time()-t0:.3f}s')
# df.head(2)

In [ ]:
# first remove your own mails
condition = df['sender'] == f'{USER_EMAIL_ID}'
df.drop(df[condition].index, inplace=True)
df[condition]

In [ ]:
df.set_index(np.arange(len(df)), inplace=True)
df.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
import json

with open('../data/label_dict.txt', 'r') as file:
    label_str = file.read()

all_labels = json.loads(label_str.replace('\'', '\"'))
all_labels

In [ ]:
import re

label_list = [key for key in all_labels.keys() if re.match('Label_[0-9]', key)]
label_list

In [ ]:
mlb = MultiLabelBinarizer(classes=label_list)
# will implement onehot later (numpy array error of not being able to create a vaguely shaped 2d array)
# for one hot

In [ ]:
labels_array = [list(st.split(',')) for st in df['labels']]
labels = mlb.fit_transform(labels_array)
labels

In [ ]:
mlb.inverse_transform(labels)

In [ ]:
df.info()

In [ ]:
df.drop(['id', 'receiver'], axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
df.drop('labels', axis=1, inplace=True)

In [ ]:
df.info()

In [ ]:
st = 'hello@mfs.iitr.ac.in'
s = st.split('@')
s_df = pd.DataFrame([st.split('@') for st in df['sender']], index=df['sender'].index)
df['sender_name'], df['sender_dept'] = s_df[0], s_df[1]
df.drop('sender', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value='')
df_f = imputer.fit_transform(df)

In [ ]:
df_f = pd.DataFrame(df_f)
df_f

In [ ]:
df.loc[df['subject'].isna() == True]

In [ ]:
df.loc[df['body'].isna() == True]

In [ ]:
df_f.loc[213]

In [ ]:
df = df_f
df[2].describe()

In [ ]:
df[3].describe()

In [ ]:
df[3].value_counts()

In [ ]:
tf_idf = TfidfVectorizer()
b = tf_idf.fit_transform(df[0])
c = tf_idf.fit_transform(df[1])

In [ ]:
enc = OneHotEncoder()
a = enc.fit_transform(df.iloc[:, 2:4])
enc.categories_

In [ ]:
a.shape #one hot

In [ ]:
b.shape #tfidf

In [ ]:
c.shape #tf_idf

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.2)

In [ ]:
from scipy import sparse

lst = [[1,2,3],[4,5,6]]
csr = sparse.csr_matrix(lst)
csr.getrow()

In [ ]:
C = pd.DataFrame(c.toarray())

In [ ]:
C

In [ ]:
labels = pd.DataFrame(labels)

In [ ]:
labels

In [ ]:
for train_index, test_index in sss.split(C, labels):
    strat_train_x = C.loc[train_index]
    strat_train_y = labels.loc[train_index]
    strat_test_x = C.loc[test_index]
    strat_test_y = labels.loc[test_index]


In [ ]:
strat_train_x = C.iloc[:900]
strat_train_y = labels.iloc[:900]
strat_test_x = C.iloc[900:]
strat_test_y = labels.iloc[900:]


In [ ]:
# clean the body of the message

import nltk
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import WordNetCorpusReader

tokenizer = nltk.RegexpTokenizer(r'\w+')